In [16]:
import pandas as pd
import os
import re

In [156]:
#code-davinci-002, vicuna-13b
MODEL_NAME = 'vicuna-13b'
CATEGORY = 'tracking_three'
DF_NAME = 'tracking_three'


In [157]:
def save_csv(row):
    file_path = SAVE_PTH+'_cleaned.csv'

    if not os.path.isfile(file_path):
        # The file does not exist, write with header
        row.to_frame().T.to_csv(file_path, mode='a', index=False, header=True)
    else:
        # The file exists, append without header
        row.to_frame().T.to_csv(file_path, mode='a', index=False, header=False)

In [158]:
for i in range(3):    
    SAVE_PTH = f'../data/adaptive_consistency_outputs/{MODEL_NAME}/{DF_NAME}/{DF_NAME}_seed{i+1}'
    df_t07 = pd.read_json(SAVE_PTH+'.jsonl', lines=True)
    save_dict = {
        'Category': [CATEGORY for i in range(len(df_t07))],
        'Question':[None for i in range(len(df_t07))],
        'Correct Answer': [None for i in range(len(df_t07))],
    }
    df_save = pd.DataFrame(save_dict)
    for row_idx in range(len(df_t07)):
        row = df_t07.iloc[row_idx]
        row_save = df_save.iloc[row_idx]
        row_save['Question']= row['input']
        right_answer = row['target']
        row_save['Correct Answer'] = right_answer
        cot_li = row['generation']
        cot_ans = row['answers']
        for idx,cot in enumerate(cot_li):
            if idx >= 40:
                break
            row_save[f'CoT_{idx}'] = cot
            row_save[f'Final Answer_{idx}'] = cot_ans[idx]
            row_save[f'Instruction Violation_{idx}'] = [(0,0)]
        save_csv(row_save)